In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# 생년월일, 이름, 태어난시간, 이름, 분석유형

In [3]:
from typing import List
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

class User_info(BaseModel):
    
    # 이름
    name: str = Field(description="Name of the analyst.")

    # 생년월일
    date: str = Field(
        description="Date of birth.",
    )

    # 태어난 시간
    time: str = Field(
        description="Time of birth",
    )
    
    # 분석 유형
    analysis: str = Field(
        description="Description of the analyst focus, concerns, and motives.",
    )

    # 사용자가 실제로 묻는 질문
    question: str = Field(description="User's question for the saju analysis.")
    
    # 유저의 인적 정보를 문자열로 반환하는 속성
    @property
    def persona(self) -> str:
        return f"Name: {self.name}\nDate: {self.date}\nTime: {self.time}\nDescription: {self.analysis}\n"
    
    

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [5]:
from langgraph.graph import END
from langchain_core.messages import HumanMessage, SystemMessage

# 유저 입력 프롬프트
analyst_instructions = """
아래 정보를 순서대로 입력해 주세요
1. 이름: {name}
2. 생년월일 (YYYY-MM-DD): {date}
3. 태어난 시간: (HH:MM) {time}
4. 분석 유형(예: 사주 전반, 궁합, 직업운, 재물운 등) {analysis}

"""

# 유저정보 생성 노드
def create_analysts(state: User_info):
    """유저 정보"""

    name = state.get("name", "")
    date = state.get("date", "")
    time = state.get("time", "")
    analysis = state.get("analysis", "")

    # 유저정보 생성을 위한 시스템 프롬프트 구성
    system_message = analyst_instructions.format(
        name=name,
        date=date,
        time=time,
        analysis=analysis
    )

    # LLM에 구조화된 출력 형식을 적용
    structured_llm = llm.with_structured_output(User_info)

    user = structured_llm.invoke(
        [SystemMessage(content=system_message)
        , HumanMessage(content="Generate the set of User_info.")]
    )

    # state["analysts"] = analysts  # 또는 analysts.dict()로 깔끔히 직렬화해도 좋습니다.

    # 유저 정보를 상태에 저장
    state["user"] = user

    return state

In [6]:
initial_state = {
    "name": "김재호",
    "date": "1995-03-28",
    "time": "12:35",
    "analysis": "사주 전반"
}

user = create_analysts(initial_state)

In [7]:
user

{'name': '김재호',
 'date': '1995-03-28',
 'time': '12:35',
 'analysis': '사주 전반',
 'user': User_info(name='김재호', date='1995-03-28', time='12:35', analysis='사주 전반', question='')}

In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def answer_question(state):
    user = state["user"]
    question = state["question"]

    # 1) PromptTemplate 으로 문자열을 Runnable로 포장
    template = PromptTemplate.from_template(
        "당신은 사주 분석 전문가입니다.\n"
        f"{user}\n"
        f"사용자 질문: {question}\n"
        "위 정보를 바탕으로 친절하고 상세하게 답변해 주세요."
    )

    # 2) LLM과 parser
    llm = ChatOpenAI(model="gpt-4o-mini")
    parser = StrOutputParser()

    # 3) RunnableSequence 구성
    chain = template | llm | parser

    # 4) invoke 시 입력 키는 template 변수명과 일치해야 함
    generation = chain.invoke({
        "user": user,
        "question": question
    })

    return {"user": user, "question": question, "generation": generation}


In [9]:
question = "오늘 제 재물운이 궁금해요"
output = answer_question({
    "user": user,   # 반드시 User_info 타입
    "question": question
})
print(output["generation"])

KeyError: 'Input to PromptTemplate is missing variables {"\'name\'"}.  Expected: ["\'name\'"] Received: [\'user\', \'question\']\nNote: if you intended {\'name\'} to be part of the string and not a variable, please escape it with double curly braces like: \'{{\'name\'}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '

In [ ]:
from langgraph.graph import StateGraph, END

graph = StateGraph(User_info)

graph.add_node("create_analysts", create_analysts)
graph.link("question", END)